In [1]:
import pandas as pd 
#from bs4 import BeautifulSoup
import requests
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from time import sleep  
pd.set_option('display.max_columns', None)

## 1. Metemos los csv de las pelis

In [7]:
data = pd.read_csv("horror_1.csv", index_col=0)  #lo único que tenemos que cambiar cada vez es el nombre del csv
data.head()

,type,name,year,month,id
genre,,,,,
Horror,Movie,The Legend of Johnny Jones,2024,10,tt10928904
Horror,Movie,Tanha Nam Man Phrai,1990,1,tt8449060
Horror,Movie,Johnny in Monsterland,1990,6,tt7097770
Horror,Movie,The Horror,1990,2,tt4299276
Horror,Movie,Kashmoraa,1990,1,tt3521170


In [8]:
len(data)

113

In [9]:
ids = data["id"] ##Esto no lo tocamos, ya está definido para coger únicamente el id de la peli que se va a leer en el buscador de IMDB
ids.head()

genre
Horror    tt10928904
Horror     tt8449060
Horror     tt7097770
Horror     tt4299276
Horror     tt3521170
Name: id, dtype: object

In [ ]:
#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1) > div.sc-bfec09a1-7.gWwKlt > a
#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(2) > div.sc-bfec09a1-7.gWwKlt > a
#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.sc-bfec09a1-7.dpBDvu > a

## 2. Extracción de los actores de las películas y guardamos la info en un csv

In [14]:
driver = webdriver.Chrome()

driver.maximize_window() #Maximizamos ventana antes de meternos en IMDB

sleep(2)

driver.get("https://www.imdb.com/")

sleep(2)

lista_actores = []

for id in tqdm(ids[0:2]):

    driver.find_element("css selector", "#suggestion-search").send_keys(id, Keys.ENTER) #Para que coja el ID de cada tupla de peli

    try: 
    
        lista_actores_consumible = []
        
        for i in range (1,11):

            try:
                nombre_actor = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.sc-bfec09a1-7.gWwKlt > a").text          
                lista_actores_consumible.append (nombre_actor)

            except:
                lista_actores_consumible.append("NA")
        
        lista_actores_consumible.append(id) #Lo pongo al nivel del "for" para que lo apendee una vez y no cada vez que itere un actor y lo añado a la lista de consumible para que aparezca dentro de la lista de tuplas de actores y tb lo pongo antes de convertir la lista a tupla (línea de código a continuación)

        lista_actores.append (tuple(lista_actores_consumible))

        driver.back()

        sleep(2)
    
    except: 
        pass

print (lista_actores)

100%|██████████| 2/2 [00:30<00:00, 15.25s/it]

[('Danny Trejo', 'Dylan Playfair', 'Clifton Powell', 'Isaac C. Singleton Jr.', 'Hector David Jr.', 'Emily Rose Hernandez', 'Aaron Cohen', 'David Siracusa', 'Bronsonn Taylor', 'Raven Armando Astone', 'tt10928904'), ('Tanit Pongmanoon', 'Somsak Chaisongkram', 'Chatrin Tana Pong', 'Nadira Al Khor', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'tt8449060')]


In [15]:
len(lista_actores)  #debería ser igual al nro de pelis en csv si encontró todos los IDs de las pelis del CSV

2

In [16]:
df_lista_actores = pd.DataFrame(lista_actores)

In [13]:
df_lista_actores.head()

,0,1,2,3,4,5,6,7,8,9,10
0,Danny Trejo,Dylan Playfair,Clifton Powell,Isaac C. Singleton Jr.,Hector David Jr.,Emily Rose Hernandez,Aaron Cohen,David Siracusa,Bronsonn Taylor,Raven Armando Astone,tt10928904
1,Tanit Pongmanoon,Somsak Chaisongkram,Chatrin Tana Pong,Nadira Al Khor,NA,NA,NA,NA,NA,NA,tt8449060


In [ ]:
df_lista_actores.to_csv("XXX_lista_actores.csv") #para guardarlo en un csv

## 3. Extracción de la información de los actores y guardamos la info en un csv

In [20]:
driver = webdriver.Chrome()

driver.maximize_window() #Maximizamos ventana antes de meternos en IMDB

sleep(2)

driver.get("https://www.imdb.com/")

sleep(2)

info_actores = []

for actores_peli in tqdm(lista_actores): #Del resultado anterior de lista de actores, vamos a iterar por cada una de las tuplas de 10 actores

    info_actores_consumible = []

    for actores in actores_peli[0:10]:  #Cogemos sólo los actores (excluyendo el ID de la peli)
    
        info_actores_consumible.append (actores.upper()) #AÑADIMOS NOMBRE en mayúsculas para verlo bien

        if actores == "NA" or "NaN":   
            info_actores_consumible.append ("NA nacimiento")
            info_actores_consumible.append ("NA conocido")
            info_actores_consumible.append ("NA profesión")
            info_actores_consumible.append ("NA oscars")


        else:

            driver.find_element("css selector", "#suggestion-search").send_keys(actores, Keys.ENTER) #busco cada actor en el "search box"

            sleep (2)
                    
            #Click en el resultado el actor para ver su información
            driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-872d7ac7-0.fqEQWL > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.iUyFfD > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a").click()
                                                 
            sleep (2)

            try:
            
            #AÑADIMOS AÑO DE NACIMIENTO      
                try:
                    nacimiento = driver.find_element ("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.dEqUUl > div.sc-e226b0e3-6.CUzkx > div.sc-e226b0e3-11.kkLqLt > section > aside > div > span:nth-child(2)").text
                    try:                                               
                        info_actores_consumible.append (nacimiento.split(",")[1]) 

                    except:
                        info_actores_consumible.append (nacimiento) 

                except:
                    info_actores_consumible.append ("NA nacimiento")
                
                sleep(2) 

                
                #AÑADIMOS POR QUÉ ES CONOCIDO           
                try:
                    conocido_lista = []
                    
                    for i in range (1,5):
                        try:
                            conocido = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a")
                            conocido_limpio = conocido.text                  
                            conocido_lista.append (conocido_limpio)
                            sleep(2)

                        except:
                            pass

                    info_actores_consumible.append(conocido_lista)  

                except:
                    info_actores_consumible.append ("NA conocido")

                sleep(2)   


                #AÑADIMOS PROFESIÓN
                try:
                    profesion = driver.find_element ("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div > ul"). text
                    try:                                              
                        info_actores_consumible.append (profesion.split("\n"))

                    except:                                               
                        info_actores_consumible.append (profesion) 
                
                except:
                    info_actores_consumible.append ("NA profesion")
                
                sleep(2) 

                #AÑADIMOS INFO SOBRE OSCARS
                try:
                    oscars = driver.find_element ("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div > ul > li > a.ipc-metadata-list-item__label.ipc-metadata-list-item__label--link").text
                    info_actores_consumible.append (oscars)             
                
                except:
                    info_actores_consumible.append ("No ha ganado ningún oscar")


            
            except:
                pass

    info_actores_consumible.append(actores_peli[10]) #Lo pongo al nivel del "for" para que lo apendee una vez y no cada vez que itere un actor y lo añado a la lista de consumible para que aparezca dentro de la lista de tuplas de actores y tb lo pongo antes de convertir la lista a tupla (línea de código a continuación)
    
    info_actores.append (tuple(info_actores_consumible))
    
    print (info_actores_consumible) #Para ir viendo que pelis se van descargando

    driver.back()

driver.close()  

print (info_actores)

 50%|█████     | 1/2 [00:00<00:00,  5.23it/s]

['DANNY TREJO', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'DYLAN PLAYFAIR', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'CLIFTON POWELL', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'ISAAC C. SINGLETON JR.', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'HECTOR DAVID JR.', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'EMILY ROSE HERNANDEZ', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'AARON COHEN', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'DAVID SIRACUSA', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'BRONSONN TAYLOR', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'RAVEN ARMANDO ASTONE', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'tt10928904']


100%|██████████| 2/2 [00:00<00:00,  9.41it/s]


['TANIT PONGMANOON', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'SOMSAK CHAISONGKRAM', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'CHATRIN TANA PONG', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'NADIRA AL KHOR', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'NA', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'NA', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'NA', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'NA', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'NA', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'NA', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'tt8449060']
[('DANNY TREJO', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'DYLAN PLAYFAIR', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'CLIFTON POWELL', 'NA nacimiento', 'NA conocido', 'NA profesión', 'NA oscars', 'ISAAC C. SINGLETON JR

In [ ]:
len(info_actores)  #debería ser igual al nro de pelis en csv

In [ ]:
df_info_actores = pd.DataFrame(info_actores) #convertimos "info_actors" a dataframe

In [ ]:
df_info_actores.to_csv("XXX_info_actores.csv") #para guardarlo en un csv